## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-20-18-52-25 +0000,nyt,Gaza Operations of Much-Criticized U.S. Aid Gr...,https://www.nytimes.com/2025/10/20/world/middl...
1,2025-10-20-18-45-02 +0000,nypost,Judge slams NYCHA tenants for presenting ‘garb...,https://nypost.com/2025/10/20/us-news/judge-sl...
2,2025-10-20-18-39-00 +0000,nypost,Diminutive accountant stands on tip-toes to te...,https://nypost.com/2025/10/20/us-news/diminuti...
3,2025-10-20-18-36-19 +0000,cbc,Here are the facts on the jewels stolen from t...,https://www.cbc.ca/news/world/louvre-art-heist...
4,2025-10-20-18-34-52 +0000,wapo,Amazon cloud outage disrupts service of major ...,https://www.washingtonpost.com/technology/2025...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2329/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
22,trump,23
33,louvre,16
135,china,12
74,new,12
0,gaza,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
196,2025-10-19-22-46-24 +0000,bbc,China will soon have a new Five Year Plan. Her...,https://www.bbc.com/news/articles/cjr045x425vo...,59
93,2025-10-20-13-22-06 +0000,nypost,Trump insists cease-fire is holding in Gaza as...,https://nypost.com/2025/10/20/us-news/presiden...,59
19,2025-10-20-18-03-33 +0000,nypost,Trump warns Hamas with chilling statement as p...,https://nypost.com/2025/10/20/us-news/trump-wa...,55
172,2025-10-20-04-01-41 +0000,nypost,Kushner and Witkoff reveal how the Trump admin...,https://nypost.com/2025/10/20/world-news/kushn...,54
212,2025-10-19-19-27-11 +0000,nypost,Secret Service investigating possible sniper’s...,https://nypost.com/2025/10/19/us-news/secret-s...,50


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
196,59,2025-10-19-22-46-24 +0000,bbc,China will soon have a new Five Year Plan. Her...,https://www.bbc.com/news/articles/cjr045x425vo...
93,59,2025-10-20-13-22-06 +0000,nypost,Trump insists cease-fire is holding in Gaza as...,https://nypost.com/2025/10/20/us-news/presiden...
74,41,2025-10-20-15-10-02 +0000,nypost,French authorities admit they ‘failed’ in Louv...,https://nypost.com/2025/10/20/world-news/louvr...
214,39,2025-10-19-19-13-36 +0000,nypost,House Dem leader Hakeem Jeffries has awkward p...,https://nypost.com/2025/10/19/us-news/hakeem-j...
182,34,2025-10-20-00-38-24 +0000,nypost,Two dead after cargo plane skids off runway in...,https://nypost.com/2025/10/19/world-news/cargo...
200,33,2025-10-19-22-12-05 +0000,nyt,"Israel Strikes Gaza and Temporarily Halts Aid,...",https://www.nytimes.com/2025/10/19/world/middl...
106,30,2025-10-20-12-15-35 +0000,startribune,Amazon cloud computing outage disrupts Snapcha...,https://www.startribune.com/amazon-cloud-compu...
86,29,2025-10-20-13-55-15 +0000,latimes,Supreme Court will decide if 'habitual drug us...,https://www.latimes.com/politics/story/2025-10...
166,27,2025-10-20-06-34-11 +0000,nypost,Trump warned Zelensky that Putin could ‘destro...,https://nypost.com/2025/10/20/world-news/trump...
212,27,2025-10-19-19-27-11 +0000,nypost,Secret Service investigating possible sniper’s...,https://nypost.com/2025/10/19/us-news/secret-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
